In [16]:
import os
import keras
from PIL import Image, ImageOps
import numpy as np
import scipy.misc
from keras.preprocessing.image import ImageDataGenerator

img_w = 300
img_h = 225
batch_size = 20
datagen_seed = 1987 
model_path = './saved_models/simple_conv_model.h5'
train_dir = './data_gen/train'

data_generator = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.20)

train_generator = \
    data_generator.flow_from_directory(train_dir, 
                                       target_size=(img_w, img_h), 
                                       shuffle=True, seed=datagen_seed,
                                       class_mode='categorical', 
                                       batch_size=batch_size, 
                                       subset="training")

validation_generator = \
    data_generator.flow_from_directory(train_dir, 
                                       target_size=(img_w, img_h), 
                                       shuffle=True, seed=datagen_seed,
                                       class_mode='categorical', 
                                       batch_size=batch_size, 
                                       subset="validation")
    
    

Found 8221 images belonging to 120 classes.
Found 2001 images belonging to 120 classes.


In [17]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(img_w, img_h, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(120, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 298, 223, 32)      896       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 149, 111, 32)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 147, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 73, 54, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 71, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 35, 26, 128)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 33, 24, 128)       147584    
__________

In [18]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc'])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        monitor='val_acc',
        save_best_only=True
    )
]
if os.path.isfile(model_path):
    model.load_weights(model_path)
    
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=30,
      callbacks=callbacks,
      validation_data=validation_generator,
      validation_steps=25)

Epoch 1/30
50/50 [==============================] - 26s 512ms/step - loss: 15.9086 - acc: 0.0130 - val_loss: 15.9489 - val_acc: 0.0105
Epoch 2/30
50/50 [==============================] - 24s 477ms/step - loss: 15.9247 - acc: 0.0120 - val_loss: 15.9489 - val_acc: 0.0105
Epoch 3/30
50/50 [==============================] - 24s 485ms/step - loss: 16.0375 - acc: 0.0050 - val_loss: 15.9489 - val_acc: 0.0105
Epoch 4/30
50/50 [==============================] - 24s 486ms/step - loss: 15.8440 - acc: 0.0170 - val_loss: 15.9489 - val_acc: 0.0105
Epoch 5/30
50/50 [==============================] - 25s 494ms/step - loss: 15.9892 - acc: 0.0080 - val_loss: 15.9489 - val_acc: 0.0105
Epoch 6/30
50/50 [==============================] - 24s 480ms/step - loss: 15.9569 - acc: 0.0100 - val_loss: 15.9489 - val_acc: 0.0105
Epoch 7/30
50/50 [==============================] - 24s 470ms/step - loss: 16.0053 - acc: 0.0070 - val_loss: 15.9489 - val_acc: 0.0105
Epoch 8/30
50/50 [==============================] - 23s